In [4]:
print("helloooo")

helloooo


In [3]:
from river.tree import HoeffdingTreeClassifier
from river import stream
from river import metrics
from river import evaluate

import time
import datetime as dt
import pandas as pd
import os
import random

import argparse

PATH_TO_DATA = "C:/Users/Bayde/Desktop/Hoeffding_Tree_Project_Data/datasets_csv"
       
    
"""
Grace period: The number of instances a leaf should observe between split attempts. Range
of values (10; 200). Step = 10.

Split Confidence: The allowable error in split decision. Range of values (0.0; 1.0). Step =
0.05.

Tie Threshold: Threshold below which a split will be forced to break ties. Range of values
(0.0; 1.0). Step = 0.05.
"""
        
        
def model_size(model):
    if str(model) == 'HoeffdingTreeClassifier':
        return model.n_nodes
    elif str(model) == 'ARFClassifier':     # In case we are interested in counting the number of nodes from all trees.
        nodes = 0
        for m in model.models:
            nodes += m.model.n_nodes
        return nodes
    return -1


def run_experiment(model, dataset):
    accuracy = metrics.Accuracy()
    start = time.perf_counter()

    X, Y = dataset[:, :-1], dataset[:, -1]
    # Downsampling the data
    # Select every 10th element starting from the first element
    # Useful during development, when you are making sure your
    #    algorithm works before doing the actual experiments.
    
    X, Y = X[::10], Y[::10]  #TODO: <--- Remember to remove downsampling

    ds = stream.iter_array(X, Y)

    for (x, y) in ds:
        yp = model.predict_one(x)
        accuracy.update(y, yp)
        model.learn_one(x, y)

    return accuracy.get(), dt.timedelta(seconds=time.perf_counter() - start).seconds, model_size(model)

def new_tree():
    # Random grace period from the range (10, 200) with step = 10
    gp = random.choice(range(10, 201, 10))

    # Random split_confidence from the range (0.0, 1.0) with step = 0.05
    sc = random.choice([i * 0.05 for i in range(21)])

    # Random tau from the range (0.0, 1.0) with step = 0.05
    t = random.choice([i * 0.05 for i in range(21)])

    new_model = HoeffdingTreeClassifier(
        grace_period=gp,
        delta=sc,
        tau=t)
    return new_model
    
    
    
    
    
def main(grace_period, split_confidence, tie_threshold):
    
    # Optional HyperParameter Arguments for base HT Classifier, default is (10,0,0)
    print(f"Grace Period: {grace_period}")
    print(f"Split Confidence: {split_confidence}")
    print(f"Tie Threshold: {tie_threshold}")
    
    datasets = ['electricity.csv', 'SEA_abrupt.csv', 'SEA_gradual.csv',
                'RTG_2abrupt.csv', 'covtype.csv']

    model_ht = HoeffdingTreeClassifier(
        grace_period=100,
        delta=0.05,  # split confidence
        tau=0.5  # tie threshold
    )
    model_ht2 = HoeffdingTreeClassifier(
        grace_period=500,
        delta=0.001,  # split confidence
        tau=0.05  # tie threshold
    )

    
    #TODO: Implement
    models = [model_ht, model_ht2]     # could add more models in here for testing

    for name in datasets: # For Each Dataset
        dataset = pd.read_csv(os.path.join(PATH_TO_DATA, name)).to_numpy()

        for model in models: # For Each Model
            result = run_experiment(model, dataset)
            print('Dataset: ', name, ' Learner: ', model, ' Result: ', result, "(Accuracy, Time,  Size)")

            
            
if __name__ == '__main__':
#     parser = argparse.ArgumentParser(
#         description='hyperparameters of the base Hoeffding Trees, this is optional and will be otherwise hard coded')

#     parser.add_argument('--gp', type=float, choices=range(10, 201, 10),
#                         help='The number of instances a leaf should observe between split attempts. Range of values (10; 200). Step = 10.', default=10)

#     parser.add_argument('--sc', type=float, choices=[i * 0.05 for i in range(21)],
#                         help='The allowable error in split decision. Range of values (0.0; 1.0). Step = 0.05.', default=0.0)

#     parser.add_argument('--t', type=float, choices=[i * 0.05 for i in range(21)],
#                         help='Threshold below which a split will be forced to break ties. Range of values (0.0; 1.0). Step = 0.05.', default=0.0)

#     args = parser.parse_args()
    
    args = argparse.Namespace(gp=10, sc=0.0, t=0.0) #TODO remove this when not using jupyter notebook, its needed so arguments dont brick the code
    main(args.gp, args.sc, args.t)


Grace Period: 10
Split Confidence: 0.0
Tie Threshold: 0.0
Dataset:  electricity.csv  Learner:  HoeffdingTreeClassifier  Result:  (0.7583848190644307, 0, 67) (Accuracy, Time,  Size)
Dataset:  electricity.csv  Learner:  HoeffdingTreeClassifier  Result:  (0.7711827007943513, 0, 9) (Accuracy, Time,  Size)
Dataset:  SEA_abrupt.csv  Learner:  HoeffdingTreeClassifier  Result:  (0.8231, 0, 73) (Accuracy, Time,  Size)
Dataset:  SEA_abrupt.csv  Learner:  HoeffdingTreeClassifier  Result:  (0.7909, 0, 11) (Accuracy, Time,  Size)
Dataset:  SEA_gradual.csv  Learner:  HoeffdingTreeClassifier  Result:  (0.8306, 0, 73) (Accuracy, Time,  Size)
Dataset:  SEA_gradual.csv  Learner:  HoeffdingTreeClassifier  Result:  (0.8304, 0, 11) (Accuracy, Time,  Size)
Dataset:  RTG_2abrupt.csv  Learner:  HoeffdingTreeClassifier  Result:  (0.4893, 5, 203) (Accuracy, Time,  Size)
Dataset:  RTG_2abrupt.csv  Learner:  HoeffdingTreeClassifier  Result:  (0.4376, 6, 15) (Accuracy, Time,  Size)
Dataset:  covtype.csv  Learner: 